In [14]:
import json
import os
import zipfile

In [18]:
base_path = "loj/"
all_json_files = [f for f in os.listdir(base_path) if f.endswith('_en.json')]
all_json_files = sorted(all_json_files)
print(len(all_json_files))
print(all_json_files[:3])

4
['#100_en.json', '#101_en.json', '#102_en.json']


In [20]:
# unzip all testdata files
for name in all_json_files[:3]:
    testfile = "loj/testdata/TestData_" + name.split("_")[0] + ".zip"
    extract_dir = os.path.splitext(testfile)[0]
    if os.path.exists(extract_dir):
        continue
    os.makedirs(extract_dir, exist_ok=True)
    
    with zipfile.ZipFile(testfile, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
        print(f"Extracted {testfile} to {extract_dir}")

In [21]:
# generate config.yaml for each testdata folder
# config.yaml:
# time: 1s
# memory: 125mb
# the time and memory can be found in the json file
for name in all_json_files[:3]:
    test_dir = "loj/testdata/TestData_" + name.split("_")[0]
    with open(base_path + name, 'r') as f:
        data = json.load(f)
        time_limit = str(int(data['labels'][1].split(" ")[0])/1000) + "s"
        memory_limit = str(int(data['labels'][2].split(" ")[0]))
        if data['labels'][2].split(" ")[1] == "MiB":
            memory_limit += "mb"
        elif data['labels'][2].split(" ")[1] == "KiB":
            memory_limit += "kb"
        elif data['labels'][2].split(" ")[1] == "GiB":
            memory_limit += "gb"
    with open(test_dir + "/config.yaml", 'w') as f:
        f.write(f"time: {time_limit}\n")
        f.write(f"memory: {memory_limit}\n")
    print(f"Generated config.yaml for {test_dir}")

Generated config.yaml for loj/testdata/TestData_#100
Generated config.yaml for loj/testdata/TestData_#101
Generated config.yaml for loj/testdata/TestData_#102


In [22]:
# generate problem.yaml for each problem
# problem.yaml:
# title: 题目名
# tag:
# - 标签1
# - 标签2
# pid: 题号（字母+数字）
for name in all_json_files[:3]:
    test_dir = "loj/testdata/TestData_" + name.split("_")[0]
    with open(base_path + name, 'r') as f:
        data = json.load(f)
        title = data['title']
        remove_str = title.split(" ")[0] + " "
        title = title.replace(remove_str, "")
        labels = data['labels']
        tag_list = []
        start = False
        for i in labels:
            if i == "Hide Tags":
                start = True
                continue
            if start:
                tag_list.append(i)
        pid = "L" + data["ID"]
    with open(test_dir + "/problem.yaml", 'w') as f:
        f.write(f"title: {title}\n")
        f.write("tag:\n")
        for i in tag_list:
            f.write(f"- {i}\n")
        f.write(f"pid: {pid}\n")
    print(f"Generated problem.yaml for {test_dir}")

Generated problem.yaml for loj/testdata/TestData_#100
Generated problem.yaml for loj/testdata/TestData_#101
Generated problem.yaml for loj/testdata/TestData_#102


In [23]:
# generate problem.md for each problem
# problem.md is the mark down file, and the input will be 
# "content": {
#         "题目描述": "ZQC 和妹子来到了一片小树林，看到一列列参差不齐的树，ZQC 想到了一种评价每一列树的美观程度的方法 —— 将相邻两棵树高度差的总和作为一列树的美观度。即，设一列树的高度组成的序列为 A ，则其美观度为\nf(A) = \\sum\\limits_{i = 1} ^ {|A| - 1} |A_i - A_{i + 1}| 。\nZQC 想，每一列树都有一些特征，于是他钦定了一列树的特征值 —— 设 S 为 A 的美观度最大的子序列，若 A 有 k 个子序列 T_i 的美观度与 S 相同（即 f(T_i) = f(S) ），则称 A 的特征值为 k 。注意，子序列不能为空。\nZQC 的妹子非常喜欢 k 这个特征值，她希望 ZQC 能给她种一列特征值为 k 的树。按照套路，这么简单的问题 ZQC 当然不会亲自出马，所以他钦定你来解决这个问题。\n给出一个序列 A 的特征值 k （ 1 \\leq k \\leq 10 ^ {18} ），要求构造一个这样的序列 A （ 1 \\leq |A| \\leq 5000 ， 0 \\leq A_i \\leq 2 ^ {31} - 1 ）。",
#         "输入格式": "一行一个数 k 。",
#         "输出格式": "如果有解，输出两行。第一行一个正整数 n 。第二行 n 个正整数表示构造的序列 A 。\n如果无解，输出一行 qnq。",
#         "样例 1": "",
#         "样例 2": ""
#     },
#  "samples": [
#         {
#             "input": "3 4 5\n-2 -8 -9 8\n-10 0 6 -8\n-10 -6 6 9\n4 -7 5 -5 9\n10 -2 -10 5 5\n-3 -7 -3 8 -2\n-6 7 7 3 -2",
#             "output": "999999898 149 153 999999929 999999951\n999999997 999999979 999999883 74 999999921\n999999835 103 55 95 999999857"
#         }
#     ],
# to:
# ## Sample Input 2
# 
# ```
# 1 1
# ```
# 
# ### Sample Output 2
# 
# ```
# Draw
# ```
#
for name in all_json_files[:3]:
    test_dir = "loj/testdata/TestData_" + name.split("_")[0]
    with open(base_path + name, 'r') as f:
        data = json.load(f)
        content = data['content']
        translated_content = {}
        for key in content.keys():
            if key == "题目描述":
                translated_content["Problem Description"] = content[key]
            elif key == "输入格式":
                translated_content["Input Format"] = content[key]
            elif key == "输出格式":
                translated_content["Output Format"] = content[key]
            elif key == "数据范围与提示":
                translated_content["Data Range and Hint"] = content[key]
            elif key.startswith("样例"):
                pass
            else:
                translated_content[key] = content[key]
        content_str = ""
        # generate markdown content
        for key in translated_content.keys():
            content_str += f"## {key}\n\n"
            content_str += translated_content[key] + "\n\n"
        content_str += "## Sample Input\n\n"
        for i, sample in enumerate(data['samples']):
            content_str += f"### Sample Input {i+1}\n\n"
            content_str += "```\n"
            content_str += sample['input'] + "\n"
            content_str += "```\n\n"
            content_str += f"### Sample Output {i+1}\n\n"
            content_str += "```\n"
            content_str += sample['output'] + "\n"
            content_str += "```\n\n"
    with open(test_dir + "/problem.md", 'w') as f:
        f.write(content_str)
    print(f"Generated problem.md for {test_dir}")

Generated problem.md for loj/testdata/TestData_#100
Generated problem.md for loj/testdata/TestData_#101
Generated problem.md for loj/testdata/TestData_#102
